# 2.1.7 Multigrid for hybrid methods

Mixed methods for second order problems can often be reduced to the mesh facet, so called hybrid mixed methods. Simiar, hybrid DG methods introduce new variables on the facets, such that the bulk of element variables can be condensed out.

We show how to setup a multigrid preconditioner for hybrid methods. Interesting applications are nearly incompressible materials, or Stokes, discretized by $H(\operatorname{div})$-conforming HDG or hybrid mixed methods.


In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.la import EigenValues_Preconditioner

The  hybrid DG method:

On $V_h = V_T \times V_F = P^k({\mathcal T}) \times P^k ({\mathcal F})$ we define the bilinear form

$$
\DeclareMathOperator{\Div}{div}
  \sum_T \int_T \nabla u \nabla v
- \sum_T \int_{\partial T} \tfrac{\partial u}{\partial n} (v-\widehat v)
- \sum_T \int_{\partial T} \tfrac{\partial v}{\partial n} (u-\widehat u)
+ \frac{\alpha p^2}{h} \sum_F \int_F (u-\widehat u)(v-\widehat v)
$$

Element variables can be condensed out, which leads to a system reduced to the Skeleton. 

When splitting a large triangle $T_H$ into small trianles, the functions on $\partial T_H$ have a canonical representation on the facets of the fine triangles. However, facet variables on internal edges of $T_H$ are not defined by embedding. The `HarmonicProlongation` provides the energy optimal extension to the internal edges. To define energy optimal we need the energy defined by a bilinear form.

In [ ]:
ngmesh = unit_square.GenerateMesh(maxh=2)
mesh = Mesh(ngmesh)

order = 3
fes = L2(mesh, order=order) * FacetFESpace(mesh, order=order, hoprolongation=True, dirichlet=".*")
(u,uhat), (v,vhat) = fes.TnT()

n = specialcf.normal(2)
h = specialcf.mesh_size
dS = dx(element_vb=BND)

HDGform = u*v*dx+ grad(u)*grad(v)*dx - n*grad(u)*(v-vhat)*dS - n*grad(v)*(u-uhat)*dS + 5*(order+1)**2/h*(u-uhat)*(v-vhat)*dS
bfa = BilinearForm(HDGform, condense=True).Assemble()
fes.SetHarmonicProlongation(bfa, inverse="sparsecholesky")
pre = preconditioners.MultiGrid(bfa, blocktype=["vertexpatch"], cycle=1)

In [ ]:
with TaskManager():
    for l in range(7):
        mesh.Refine()
        bfa.Assemble()
        # pre.Update()
        lam = EigenValues_Preconditioner(bfa.mat, pre)
        print ("l =", l, "ndof =", fes.ndof, "lam_min/lam_max = ", lam[0], lam[-1])

In [ ]:
f = LinearForm (x*v*dx).Assemble()
gfu = GridFunction(fes)
gfu.vec[:]=0
with TaskManager():
    Solve (bfa*gfu==f, pre, lin_solver=solvers.CGSolver, printrates=True)

In [ ]:
Draw (gfu.components[0]);

## Hybrid-mixed methods:

Find $\sigma, u, \widehat u \in \Sigma_h \times V_h \times F_h$:

$$
\DeclareMathOperator{\Div}{div}
\begin{array}{ccccccll}
\int a \sigma \tau & + & \sum_T \int_T \Div \tau \, u & + & \sum_F \int_F [\tau_n] \widehat u & = & 0 & \forall \, \tau \in \Sigma \\
\int \Div \sigma \, v &&&&& = & \int f v & \forall \, v \in V_h \\
\int [ \sigma_n ] \, \widehat v &&&&& = & \int_{\Gamma_n} g \widehat v & \forall \, \widehat v \in F_h
\end{array}
$$

where $\Sigma_h$ is an discontinuous $H(div)$ finite element space, $V_h$ a sub-space of $L_2$, and $F_h$ consists of polynomials on every facet.

In [ ]:
ngmesh = unit_square.GenerateMesh(maxh=0.2)
mesh = Mesh(ngmesh)

order = 2

fesSigma = PrivateSpace(Discontinuous(HDiv(mesh, order=order, RT=True)))
fesL2 = L2(mesh, order=order)
fesFacet = FacetFESpace(mesh, order=order, hoprolongation=True, dirichlet=".*")
fes = fesSigma*fesL2*fesFacet

(sigma, u,uhat), (tau, v,vhat) = fes.TnT()
n = specialcf.normal(2)
dS = dx(element_vb=BND)
mixedform = -sigma*tau*dx
mixedform += div(sigma)*v*dx - sigma*n*vhat*dS 
mixedform += div(tau)*u*dx - tau*n*uhat*dS 

bfa = BilinearForm(mixedform, condense=True).Assemble()
fes.SetHarmonicProlongation(bfa, inverse="sparsecholesky")
pre = preconditioners.MultiGrid(bfa, blocktype=["vertexpatch"], cycle=1)

for l in range(5):
    mesh.Refine()
    bfa.Assemble()
    lam = EigenValues_Preconditioner(bfa.mat, pre)
    print ("l =", l, "ndof =", fes.ndof, "lam_min/lam_max = ", lam[0], lam[-1])

In [ ]:
f = LinearForm(v*1*dx).Assemble()
gfu = GridFunction(fes)
Solve(bfa*gfu==f, pre, solvers.CGSolver, printrates=True)
Draw (gfu.components[1]);

## Nearly incompressible materials, H(div)-conforming HDG

[Lehrenfeld+Schöberl, 2016]

In [ ]:
ngmesh = unit_square.GenerateMesh(maxh=0.3)
mesh = Mesh(ngmesh)

order = 3

fesT = HDiv(mesh, order=order, hoprolongation=True, dirichlet=".*")
fesF = TangentialFacetFESpace(mesh, order=order, hoprolongation=True, highest_order_dc=True, dirichlet=".*")
fes = fesT*fesF

(u,uhat), (v,vhat) = fes.TnT()
n = specialcf.normal(2)
def tang(v): return v-(v*n)*n
h = specialcf.mesh_size
dS = dx(element_vb=BND)

HDGform = InnerProduct(Grad(u),Grad(v))*dx - (Grad(u)*n)*tang(v-vhat)*dS - (Grad(v)*n)*tang(u-uhat)*dS \
    + 1*(order+1)**2/h*tang(u-uhat)*tang(v-vhat)*dS

bfa = BilinearForm(HDGform + 1e3*div(u)*div(v)*dx, condense=True).Assemble()
fes.SetHarmonicProlongation(bfa)
pre = preconditioners.MultiGrid(bfa, smoother="block", smoothingsteps=1, blocktype=["vertexpatch"], cycle=1)

In [ ]:
with TaskManager():
    for l in range(4):
        mesh.Refine()
        bfa.Assemble()
        lam = EigenValues_Preconditioner(bfa.mat, pre)
        print ("l =", l, "ndof =", fes.ndof, lam[0], lam[-1])

In [ ]:
with TaskManager():
    f = LinearForm ((0.5-y)*v[0]*dx).Assemble()
gfu = GridFunction(fes)
gfu.vec[:]=0
with TaskManager(pajetrace=10**8):
    solvers.BVP(bfa, f, gfu, pre, print=True)

## Nearly incompressible materials / Stokes in 3D

In [ ]:
ngmesh = unit_cube.GenerateMesh(maxh=0.3)
mesh = Mesh(ngmesh)

order = 2

fesT = HDiv(mesh, order=order, hoprolongation=True, dirichlet=".*")
fesF = TangentialFacetFESpace(mesh, order=order, hoprolongation=True, highest_order_dc=True, dirichlet=".*")
fes = fesT*fesF

(u,uhat), (v,vhat) = fes.TnT()
n = specialcf.normal(3)
def tang(v): return v-(v*n)*n
h = specialcf.mesh_size
dS = dx(element_vb=BND)

HDGform = 0.001*u*v*dx+InnerProduct(Grad(u),Grad(v))*dx - (Grad(u)*n)*tang(v-vhat)*dS - (Grad(v)*n)*tang(u-uhat)*dS \
    + 5*(order+1)**2/h*tang(u-uhat)*tang(v-vhat)*dS

bfa = BilinearForm(HDGform + 1e3*div(u)*div(v)*dx, condense=True).Assemble()
fes.SetHarmonicProlongation(bfa)
pre = preconditioners.MultiGrid(bfa, smoother="block", smoothingsteps=3, blocktype=["edgepatch"], cycle=1)

In [ ]:
with TaskManager():
    for l in range(2):
        mesh.Refine()
        bfa.Assemble()
        # pre.Update()
        lam = EigenValues_Preconditioner(bfa.mat, pre)
        print ("l =", l, "ndof =", fes.ndof, "lam min/max = ", lam[0], lam[-1])

In [ ]:
with TaskManager(pajetrace=10**8):
    f = LinearForm ((0.5-y)*v[0]*dx).Assemble()
gfu = GridFunction(fes)
gfu.vec[:]=0

with TaskManager():
    Solve(bfa*gfu==f, pre, solvers.CGSolver, printrates=True)

In [ ]:
clipping = { "function" : True,  "pnt" : (0.5,0.5,0.5), "vec" : (0,0,-1) }
Draw (gfu.components[0], order=2, clipping=clipping);

## Flow channel in 3D

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
from ngsolve.krylovspace import CGSolver

box = Box((0,0,0), (2.5,0.41,0.41))
box.faces.name="wall"
box.faces.Min(X).name="inlet"
box.faces.Max(X).name="outlet"
cyl = Cylinder((0.5,0.2,0), Z, h=0.41,r=0.05)
cyl.faces.name="cyl"
shape = box-cyl

mesh = shape.GenerateMesh(maxh=0.2).Curve(3)
Draw (mesh);

In [ ]:
order = 2

fesT = HDiv(mesh, order=order, hoprolongation=True, dirichlet="wall|inlet|cyl")
fesF = TangentialFacetFESpace(mesh, order=order, hoprolongation=True, highest_order_dc=True, dirichlet=".*")
fes = fesT*fesF

(u,uhat), (v,vhat) = fes.TnT()
n = specialcf.normal(3)
def tang(v): return v-(v*n)*n
h = specialcf.mesh_size
dS = dx(element_vb=BND)

HDGform = 0.001*u*v*dx+InnerProduct(Grad(u),Grad(v))*dx - (Grad(u)*n)*tang(v-vhat)*dS - (Grad(v)*n)*tang(u-uhat)*dS \
    + 5*(order+1)**2/h*tang(u-uhat)*tang(v-vhat)*dS

with TaskManager(pajetrace=10**9):
    bfa = BilinearForm(HDGform + 1e3*div(u)*div(v)*dx, condense=True).Assemble()
    fes.SetHarmonicProlongation(bfa)
    pre = preconditioners.MultiGrid(bfa, inverse="sparsecholesky", smoother="block", smoothingsteps=1, blocktype=["edgepatch"], cycle=1)

In [ ]:
with TaskManager():
    for l in range(1):
        mesh.Refine()
        # bfa.Assemble()
        # lam = EigenValues_Preconditioner(bfa.mat, pre)
        # print ("l =", l, "ndof =", fes.ndof, "lam min/max = ", lam[0], lam[-1])

In [ ]:
gfu = GridFunction(fes)

uin = (1.5*4*y*(0.41-y)/(0.41*0.41)*z*(0.41-z)/0.41**2,0, 0)
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

inv = CGSolver(bfa.mat, pre.mat, printrates=True, tol=1e-5)

with TaskManager(pajetrace=10**8):
    gfu.vec.data -= inv@bfa.mat * gfu.vec
gfu.vec.data += bfa.harmonic_extension * gfu.vec

In [ ]:
clipping = { "function" : True,  "pnt" : (1,0.2,0.2 ), "vec" : (0,0,-1.0) }
Draw (gfu.components[0], mesh, order=2, clipping=clipping);